# Урок 6. Математическая оптимизация

## 1. Знакомство с пакетами
Одним из самых популярных пакетов для решения задач математической оптимизации является [JuMP](https://jump.dev/JuMP.jl/stable/). 
  
Данный пакет использует различные варианты solver'ов, Вы можете ознакомиться с полным списком и возможностями каждого [тут](https://jump.dev/JuMP.jl/stable/installation/). 
Сегодня мы будем использовать: 
- [GLPK](https://github.com/jump-dev/GLPK.jl), Солвер написанный на С для решения задач линейного (Linear programming) и целочисленного (Mixed-integer linear programming) программирования
- [SCS](https://github.com/jump-dev/SCS.jl), использующий метод сопряженных конусов. 

Установите их, если не делали это ранее:

In [ ]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("Convex")
#Pkg.add("GLPK")
#Pkg.add("SCS")

## 2. Задача линейного программирования

В качестве примера, рассмотрим простейшую задачу линейного программирования: 
Мы решили открыть кофейню, которая делает два вида кофе "Раф кофе" за 400 рублей и  "Капучино" за 300.  

Чтобы сварить 1 чашку Раф кофе необходимо: 40 гр. зерен, 140 гр. молока и 5 гр. ванильного сахара. 
Для того чтобы получить одну чашку капучино нужно потратить: 30 гр. зерен, 120 гр. молока
На складе есть: 500 гр. зерен, 2000 гр. молока и 40 гр. ванильного сахара. 

Необходимо найти план варки кофе, обеспечивающий максимальную выручку от их реализации, учитывая, что нам не важно мнение покупаетелей, главное потратить весь ванильный сахар.

In [ ]:
using JuMP
using GLPK

JuMP позволяет создавать индексированные массивы с помощью метода `Containers`, подробнее [тут](https://jump.dev/JuMP.jl/dev/containers/). Зададим цену на напитки

In [ ]:
drinks = ["Раф кофе", "Капучино"]
ingredients = ["Кофе в зернах", "Молоко", "Ванильный сахар"]

cost = JuMP.Containers.DenseAxisArray(
    [400, 300],
    drinks)

Запишем рецепты

In [ ]:
drinks_receipt = JuMP.Containers.DenseAxisArray(
    [40 140 5;
     30 120 0], 
    drinks, 
    ingredients)

Зададим ограничения на кол-во ингредиентов

In [ ]:
constr_ingredients = JuMP.Containers.DenseAxisArray(
    [0 500;
     0 2000;
     40 40], # Помните условие про "потратить весь ванильный сахар" Ю 
    ingredients, 
    ["min", "max"])

Назначим solver и создадим модель:

In [ ]:
model = Model(GLPK.Optimizer)

Зададим переменные и ограничения. В `plan_ingredients` будем складывать план по затраченным ресурсам, а в `plan` – план варки 

In [ ]:
@variables(model, begin
    constr_ingredients[i, "min"] <= plan_ingredients[i = ingredients] <= constr_ingredients[i, "max"]
    plan[drinks] >= 0
end)

План варки и затраченные ресурсы должны учитывать ограничение в виде соответствия рецептуре. Зададим такое ограничение: 

In [ ]:
@constraint(model, [c in ingredients],
    sum(drinks_receipt[d, c] * plan[d] for d in drinks) == plan_ingredients[c]
)

В качестве целевой функции исполбьзуем максимизацию выручки:

In [ ]:
@objective(model, Max, sum(cost[d] * plan[d] for d in drinks))


Метод `optimize!` запускает поиск оптимального решения

In [ ]:
JuMP.optimize!(model)
term_status = JuMP.termination_status(model)
println("Статус: $term_status")

In [ ]:
hcat(plan.data,JuMP.value.(plan.data))

In [ ]:
hcat(plan_ingredients.data, JuMP.value.(plan_ingredients))

## 3. Выпуклая оптимизация

Для решения задач выпуклой оптимизации мы будем использовать пакет [Convex](https://jump.dev/Convex.jl/stable/), созданый специально для этого и поддерживающий [Disciplined Convex Programming](https://dcp.stanford.edu/home). 

Рассмотрим пример [отсюда](https://jump.dev/Convex.jl/stable/quick_tutorial/):

\begin{aligned}
\begin{array}{ll}
\text{minimize} & \|Ax - b\|_2^2 \\
\text{subject to} & x \geq 0
\end{array}
\end{aligned}

где $x\in \mathbf{R}^{n}$, $A \in \mathbf{R}^{m \times n}$, $b \in \mathbf{R}^{m}$.



In [ ]:
using Convex, SCS

Сгенерируем данные для задачи

In [ ]:
m = 4;  n = 5
A = randn(m, n); b = randn(m, 1)

Создадим искомый вектор

In [ ]:
x = Variable(n)

Зададим целевую функцию и ограничение (оно одно)

In [ ]:
problem = minimize(sumsquares(A * x - b), [x >= 0])

С помощью метода `solve!` запустим решение

In [ ]:
solve!(problem, () -> SCS.Optimizer(verbose=true))

In [ ]:
println(problem.status)
println(problem.optval)
println(x.value)

## 4. Оптимальная рассадка по залам
Вдохновлено примером [отсюда](https://jump.dev/Convex.jl/stable/examples/mixed_integer/section_allocation/)

Представьте, что мы хотим провести DataFest с 5 разными секциями и у для каждой из них мы забронировали 5 залов различной вместимости: в каждом зале не должно быть меньше 180 и больше 250 людей, а на третьей секции активность подразумевает, что должно быть точно 220 человек.

При этом организаторам удалось собрать 1000 заявок с указанием приоритета посещения 3 трех секций, где 1 - максимальный приоритет, 3 минимальный, а 10000 означает, что человек не пойдет на эту секцию. 

Наша задача дать рекомендацию слушателю, на какую же секцию ему все таки пойти, чтобы хватило места всем. 

Для начала сгенерируем матрицу приоритетов:

In [ ]:
using Random
s = [1, 2, 3, 1e4, 1e4]' # Типичные приоритеты
priority = shuffle(s) # Пошафлим их
for i = 1:999
    priority = vcat(priority, shuffle(s))
end

Создадим матрицу рассадки, в которой строки будут соответствать слушателю, а столбцы секциям:

In [ ]:
X = Variable(size(priority), :Bin)

Наложим ограничения

In [ ]:
constraints = [sum(X, dims=2) == 1, sum(X, dims=1) <= 250, sum(X, dims=1) >=180, sum(X, dims=1)[:, 3] == 220]

Целевой функций для минимизации будет ни что иное как $X^T \times P$, где $X$ - Матрица рассадки, а $P$ - матрица приоритетов

In [ ]:
p = minimize(vec(X)' * vec(priority), constraints)

solve!(p, GLPK.Optimizer)

In [ ]:
sum(X.value, dims=1)

In [ ]:
p.optval